In [2]:
# import tensorflow as tf
import numpy as np
import pandas as pd
# from dnn_tau import Dnn_tau
import sys
sys.path.append('./utils/')
from DD_data_extractor_git import Data_extractor_v4,Data_extractor_v5, output_vars_v4, normalize, bucketize, split_dataset2, flatten_2D_list, output_vars_v5, split_dataset_multitrain
import os
import fnmatch
# from utils import 
from copy import deepcopy
import pickle
from tqdm.notebook import tqdm




In [3]:
Data_saveName = 'multitrain_Sep8'

In [4]:
path = "/data/hnl/prompt_tau/anatuple/nanoV10/TEST10/"
# features=['1_eta', '1_mass', '1_phi', '1_pt', '2_eta', '2_mass', '2_phi', '2_pt', '3_eta', '3_mass', '3_phi', '3_pt', 'MET_phi', 'MET_pt']
# features = ['eta_1', 'mass_1', 'phi_1', 'pt_1', 'eta_2', 'mass_2', 'phi_2', 'pt_2', 'eta_3', 'mass_3', 'phi_3', 'pt_3', 'phi_MET', 'pt_MET']
features=[]
features_out = deepcopy(output_vars_v5)
features.extend(features_out)
features.extend(['signal_label', 'channel', 'event_type', 'mass_hyp'])
# channels = os.listdir(path)
channels = ['tee', 'tem', 'tmm', 'tte', 'ttm']
relative_path = "/anatuple/"
flat_features = flatten_2D_list(features)
print(flat_features)

['event', 'genWeight', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'phi_1', 'phi_2', 'phi_3', 'phi_MET', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'mass_12', 'mass_13', 'mass_23', 'mass_123', 'Mt_tot', 'HNL_CM_a

In [5]:
values = []
for i in range(len(flat_features)):
    values.append([])
data = dict(zip(flat_features, values))


for channel in tqdm(channels, desc='channels'):
    extractor = Data_extractor_v5(channel)
    data_currchannel = extractor(path+channel+relative_path, data=data)
    # print(data_currchannel.keys())
    for key in data.keys():
        data[key].extend(data_currchannel[key])



channels:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
data_dict = data
print(data_dict.keys())
print(np.unique(data_dict['signal_label']))
print(np.unique(data_dict['channel']))

dict_keys(['event', 'genWeight', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'phi_1', 'phi_2', 'phi_3', 'phi_MET', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'mass_12', 'mass_13', 'mass_23', 'mass_123', 'Mt_tot',

In [7]:
data = pd.DataFrame(data_dict)
# data = data.rename(columns={"genWeight": "weightOriginal"})
weightNorm = deepcopy(data['genWeight'])
data['weightNorm'] = weightNorm

In [8]:
N = len(data['event'])
data_norm = normalize(pd.DataFrame(data), 'mass_hyp', N, weight_name='weightNorm')
data_norm = normalize(data_norm, 'signal_label', N, weight_name='weightNorm')
data_norm = normalize(data_norm, 'channel', N/5, weight_name='weightNorm')
data_processed, channel_indices = bucketize(data_norm, 'channel')
print(list(data_processed.keys()))
print(channel_indices)


['event', 'genWeight', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'phi_1', 'phi_2', 'phi_3', 'phi_MET', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'mass_12', 'mass_13', 'mass_23', 'mass_123', 'Mt_tot', 'HNL_CM_a

In [9]:
current_dir = os.getcwd()
print(current_dir)

output_dir = os.path.join(current_dir,"saved_files", "extracted_data")
os.makedirs(output_dir, exist_ok=True)
data_processed.to_pickle(output_dir + "/TEST10_data_"+ Data_saveName)


/home/ddemler/HNLclassifier


In [10]:
singletrain = False
if singletrain:
    train, val, test = split_dataset2(data_processed)
    pd.to_pickle(train, output_dir + "/TEST10_train_" + Data_saveName)
    pd.to_pickle(val, output_dir +"/TEST10_val_" + Data_saveName)
    pd.to_pickle(test,output_dir +  "/TEST10_test_" + Data_saveName)
else:
    train1, train2, val1,val2 = split_dataset_multitrain(data_processed)
    pd.to_pickle(train1, output_dir + "/TEST10_train1_" + Data_saveName)
    pd.to_pickle(train2, output_dir + "/TEST10_train2_" + Data_saveName)
    pd.to_pickle(val1, output_dir +"/TEST10_val1_" + Data_saveName)
    pd.to_pickle(val2, output_dir +"/TEST10_val2_" + Data_saveName)

Total number of events: 1798683
Train1 set: 40.00 %
Train2 set: 40.00 %
Validation1 set: 10.00 %
Validation2 set: 10.00 %
